# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [9]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [10]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [11]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
#last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
last_height = browser.execute_script("return document.body.scrollHeight") #取得scroll Height
while True:
    '''
    Your code here
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # 模擬網頁下拉
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME) #等待頁面載入
    new_height = browser.execute_script("return document.body.scrollHeight") #取得新的scroll Height
    
    if last_height == new_height: #若無法往下拉就break
        break

    last_height = new_height
    print('網頁更新中')
        
print('到達頁面底端')  
    
    
    

網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [12]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8' #python 解碼由Byte Code 轉成機械碼，需Encoding
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [13]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 台積電淪外資提款機　謝金河：國安基金護盤先盯台積電

● 謝金河／《財訊》社長兼發行人、財信傳媒董事長
全球股市在疫情升溫中，就像山上石頭滾下來一般，除了中國股市外，幾乎沒有一個能倖免。
台股殺破10年缐，最低跌至8523.63，單是3月以來，台股已重挫2610.38，跌幅高達23.12%，這個跌幅跟全球大致同步，這恐怕是台股史上最慘烈的下跌。
在今年新春紅盤，台股跳空大跌近700點後，在後續的多空交戰中，稍有猶豫的投資人都難逃這一波殺戮。而且，在殺戮中，大家只求套現，根本不在乎公司有沒有價値。
這兩天，友達董事長彭雙浪跳出來喊話說，友達帳上現金逾800億，但股價跌到6.39元，市值只剩615億。
台積電穩　台股才會穩
不過，台股能不能穩得住，台積電角色非常重要。
這一波外資把台股當提款機，賣超台股114.5億美元，約3500億台幣，而外資更把台積電當超級提款機。今年迄今，外資賣超台積電705652張，大約是二千多億台幣，約是外資賣超台股的六成比重，這兩天美股狂殺，TSMC adr最低殺到43.23美元，後續TSMC在美股交易會不會再破底？也決定台積電的股價。
國安基金要護盤，最好先盯住台積電股價！ 
熱門點閱》
► 2020股災》單驥／股市、油市、疫情與貿易戰下的風暴
► 劉大年／防疫拼經濟　振興券如何發揮效用？
►  隨時加入觀點與討論，給雲論粉絲團按個讚！
 
 
----------------------------------------------------------------------

[1] 振興經濟超前部署　金融研訓院建議　恢復發行乙種建設公債

▲金融研訓院。（圖／資料照）
記者吳靜君／台北報導
新冠肺炎（COVID-19）疫情蔓延全球，全球經濟也遭遇到重擊，金融研訓院今（20）日表示，台灣在防疫的超前部署上，已經取得明顯的好成績，在此好基礎之下，思考振興經濟也要「超前部署」，建議中長期發展上，可以考慮恢復乙種建設公債，讓民眾有感經濟復甦。
金融研訓院表示，除短期因應措施之外，產業重建復興已經很難用單次貨幣政策救市，各國寬鬆性貨幣政策實施已久，多次嘗試縮表卻少有成就，所以空間小且已顯疲

● 劉大年／中華經濟研究院區域發展研究中心主任
新冠肺炎目前已蔓延全球165國，確診人數突破20萬，死亡超過萬人，已經成為本世紀全球最大的挑戰。
事實上新冠肺炎在中國大陸爆發初期，中國大陸沒有預警，資訊不夠透明；世界衞生組織（WHO)過於輕忽，全世界也掉以輕心，沒有做預先的防範，導致疫情蔓延到美國及歐洲。
2020疫情衝擊　堪比1930年代經濟大恐慌
此次新冠肺炎疫情產生供給、需求和金融三方向衝擊，對經濟影響是多層面的；更嚴重的是，由於恐慌加上對疫情的不確定性，造成市場信心瓦解，對於全球經濟更是雪上加霜。
美國財政部長最近表示，新冠肺炎對經濟的衝擊，高於2008年的全球金融海嘯，甚至直接與1930年代美國經濟大恐慌相比。德國總理也直言，德國已面臨戰後最大的危機。
▲新冠肺炎對經濟的衝擊，直接與1930年代美國經濟大恐慌相比。（圖／路透社）
事實上，美國股市大跌牽動全球股市下跌的骨牌效應，對於金融業的衝擊並不低於2008年的金融風暴。美國最近動作頻頻，聯邦準備理事會兩度大幅度降息，美國目前幾乎已是零利率。
另外，美國準備直接發給每位人民1000美元，將推出1兆美元的援助方案。手筆之大，前所未有，可見情況之嚴峻。
台灣以出口為導向　疫情擴散衝擊經濟
台灣目前防疫工作做得成功，但是由於台灣是個高度出口導向的國家，疫情擴散對於台灣經濟衝擊會增加。
疫情初期，是集中在中國大陸、日本及韓國，對台灣影響主要是供應鏈的完整性以及對一些內需產業的衝擊。但是，由於疫情已經蔓延到歐洲跟美國，這兩個地區GDP合占全球50%，一旦經濟停擺，對台灣出口會產生直接的衝擊。
以台灣出口主力─資訊電子產業為例，透過臺灣接單，大陸製造銷往歐美的三角貿易方式，疫情將使企業遭受生產面、需求面雙重衝擊。
▲疫情衝擊一個多月後，大陸華南地區、長三角一帶陸續復工。（圖／翻攝 新華社）
在生產面上，廠商在面臨原材料斷鏈、訂單混亂、物流不順的情況，即便中國大陸已逐漸復工，但產能尚無法完全恢復。
在需求面上，隨著歐美疫情擴大，造成股、匯、債市震盪，消費信心不足，再加上鎖國等防疫措施，民眾無法外出消費，將延遲民眾替換消費性電子產品的需求。另外，其他出口到歐美比重較高的汽車零組件、機械及鋼鐵產業，也會受到不同程度的影響。
外銷受阻　須避免農產品崩盤
在新冠肺炎爆發之前，原先預估今年全球經濟成長率可達到3%以上，但是由於疫

▲台銀總行。（圖／記者吳靜君攝）
記者吳靜君／台北報導
中央銀行自今（20）日起調降重貼現率、擔保放款融通利率及短期融通利率各1碼（0.25）個百分點。台銀、土銀為配合中央銀行貨幣政策，並反映當前市場利率走勢，經衡酌市場競爭及資金狀況，將自下周一（23日）起，調整利率，一年期定期儲蓄存款機動利率從1.09%、固定利率1.07%，分別調降到0.84%和0.79%。
以存款100萬元、整存整付、複利息來計算，原本一年的利息為10955元，調降後一年的利息僅剩8432元，一年減少2523元。
台銀表示，同時也調降新台幣各天期一般存款及500萬元以上定期存款及定期儲蓄存款牌告利率，調整幅0.06到0.30個百分點，並調降活期性存款利率，調整幅度0.01到0.10個百分點。土銀也表示，「一般」定期性存款各天期機動利率均調降，調降幅度0.25個百分點到0.295個百分點，同樣是下周一起調整。
▼台銀存款新牌告利率。（圖／台銀提供）
----------------------------------------------------------------------

[21] 台指期盤中大漲810點　分析師：逆價差縮小、盤勢止穩反彈機會大

▲台指期3月20日走勢。（圖／取自期交所）
記者余弦妙／台北報導
國安基金宣示進場護盤有效，台指期今（20）日開盤受到激勵大漲206點，來到8562點，接著持續上揚，盤中甚至突破9100點，來到今日高點9,166點，大漲810點大幅縮小逆價差，尾盤雖有小幅拉回，不過終仍以9013點作收，上漲657點，與現貨相較，逆價差221.09點。展望後市，萬寶投顧執行長王榮旭分析，市場出現止穩甚至扭轉，建議投資人不要做空，應該偏多操作。
王榮旭表示，今天外資在期現貨都進行調節，但調節力道也減輕很多，外資賣219億台股還能夠上漲，這也意味著國安基金進場有效，因此在外資想做空做不了的情況之下，不排除下周有可能會反過來做多，且今天的逆價差也大幅縮小。
王榮旭說，外資現在賣也是因應美國基金的贖回，因此也是被動式的賣超台股，且今天亞洲股市全面大漲，理論上外資今天應該不會再賣超，若是未來美國股市能夠止穩，基金贖回的壓力就會減少，這樣外資就會轉買，外資一旦轉買台指期就有機會反彈更大。
但王榮旭也透露，今天台指期的整體盤勢也開始扭轉，主要是因為外資賣超但仍舊大漲

▲聯華實業目前是全台規模最大、產量最高的老字號麵粉廠，能依照客戶生產製程與產品特性，客製化超過150種以上的專用粉。圖為聯華投控總經理林信宏。（圖／記者林淑慧攝）
記者林淑慧／台北報導
全台麵粉供應龍頭企業聯華實業今（20）日宣佈，將與鼎泰豐麵粉供應商僑泰興簽訂合作意向書，規劃投資聯華製粉公司股權，以強化麵粉事業營運規模與生產效率，強強聯手可望提升市佔率至近三成，未來不排除規劃上市。
聯華實業投控目前為全台產量最大的麵粉廠，年銷售量超過2億公斤。它是繼南僑後，第二家轉型為控股公司的食品公司，市場解讀此舉有利營運彈性，集團亦加速旗下子公司上市櫃規劃。
全台麵粉供應龍頭企業聯華實業去年九月轉型為控股公司，旗下事業分割為麵食、租賃及電子事業，總經理林信宏指出，麵食事業透過分割成立聯華製粉食品公司，董事會並通過與僑泰興進行策略合作可行性討論。
林信宏指出，聯華實業轉型為投資投股公司後，集團所跨足之各領域事業將採各自獨立發展政策，以使各事業體可專注深耕於其領域，與策略合作夥伴進行業務合作、合資或參股等更具靈活發展。
僑泰興於1953年成立，以嘉禾牌麵粉聞名餐飲業界，為國內知名老店如鼎泰豐等指定使用之麵粉品牌，為國內第一家獲得SQF食品安全最高等級驗證之專業麵粉廠，主要麵粉產品有「藍菊花牌」、「劍蘭牌」、「黃僑牌」等品牌。
----------------------------------------------------------------------

[34] 衛生紙原物料供貨一切正常　業者對瘋搶行為「無法理解」！

▲連鎖超市衛生紙被搶空。（圖／資料照）
記者余弦妙／台北報導
新冠肺炎疫情持續增溫，台灣確診案例已經突破百例，引起不少民眾恐慌，趕著去搶購物資，其中衛生紙更是成為大多數民眾搶購目標，甚至有賣場為確保消費者購物權益，祭出限購規則。對此，國內衛生紙廠商對於搶購行為表示「不明所以」。
正隆表示，目前衛生紙生產狀態都是正常，原物料的庫存量也都足夠，可以說面對民眾的搶購潮完全可以應付得來，只是目前就算生產的出來，但在物流配送方面遠遠趕不及民眾搶購速度，因此在生產端與消費端就出現的時間差，進而發生賣場架上沒有衛生紙的狀況。
另外，永豐餘說，目前公司生產狀態都一如既往，並沒有任何改變，也無出現衛生紙供應不足，或是原物不足的情況。對於現在市場的搶購情況則表示，這應該

▲口罩國家隊陸續提升產能，助攻口罩日均產能達到1000萬片以上。（圖／記者湯興漢攝）
記者林淑慧／台北報導
口罩國家隊日夜趕工「你累了嗎？」，有藥師在粉專發文，指出近日發現口罩瑕疵逐漸增加，經濟部今（20）日回應表示，已提醒藥局、超商等通路，瑕疵口罩先不出售，網路預購的民眾也能到超商換貨，日後會再統一回收。
有藥師在《北大藥局》粉專PO文指出，近幾天發現口罩瑕疵有增加趨勢，除了鋼條脫落、繩子貼反、還有耳掛線斷及出現髒汙的情況，藥師無奈地說，直接少掉八包的量，「瑕疵品會影響到每天實際能發放的數量，請大家多包容」。
對此，經濟部工業局官員指出，工業局在每一個口罩工廠均有駐廠人員，已經要求駐廠同仁轉達口罩生產者，應注意品質的管控，也謝謝藥房藥師在第一缐的辛苦。
另外，由經濟部斥資加碼的第二批32條新產線，原本表訂將在三月底組裝完成，但在口罩國家隊熱血投入超過2000人次趕工下，今日可望提前完工，並推升口罩日均產能上看1200萬片。
----------------------------------------------------------------------

[44] 台股強勢攻上9200點！大立光、聯發科攻漲停　指數暴衝570點

▲ 中央銀行、金管會及國安基金救市三支箭齊發，台股今日一掃陰霾應聲大漲，盤中漲點570點，收復9200點關卡。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台股今（20）日開盤後一路飆升，連續收復9000、9100與9200點關卡，漲點擴大到500點以上，目前來到9251點、上漲570點。
股王大立光（3008）強勢攻漲停，目前來到3575元；聯發科（2454）更是自10點左右就在漲停板附近狹幅震盪，目前也維持漲停價位，為301元。
另外，台積電（2330）目前亦強漲8.67%、來到270元；鴻海（2317）也收復70元大關，目前在70.9元、上漲6.49%
看到漲跌家數，目前上市總共有127家亮燈漲停，763家維持紅盤；28家下跌，跌停則為3家。
----------------------------------------------------------------------

[45] 國際原油期貨今天續漲！西德州上漲4.25%　布蘭特上漲2.14%

▲國際原油價格上漲。（圖／路透）
記者陳心怡／綜合報導
面對沙烏

▲東森購物捐贈防疫物資，向辛苦的警消醫護、疫苗研發團隊與其家人致敬。（圖／東森購物提供）
記者洪菱鞠／台北報導
為了向守護全民健康的「台灣隊長」包括醫護警消、疫苗研發團隊與其家人致敬，東森購物電話行銷團隊精選目前通路最暢銷的保健品和保養品，邀請供應商共襄盛舉，不計成本全數捐贈予第一線醫護人員，希望能為他們及家人補充健康好體力、維持美麗好氣色，而此次贊助物資總金額高達新台幣1,216萬元。
東森購物表示，此次捐贈防疫物資包括東森自然美贊助的「自然美－瑞昇超逆齡賦活膠囊」及「東森自然美瑞昇超逆齡賦活膠囊」，東森自然美執行長林燕玲觀察到，女性醫護人員因長時間戴口罩，導致臉部肌膚變得容易乾癢、脫皮或泛紅，希望她們在辛苦工作之餘，也能收到讓肌膚保持最佳狀況的優質MIT保養產品，自然散發美麗好氣色。
在新冠肺炎疫情影響下，民眾降低出門頻率，電商平台此時成為購物安全考量首選，東森購物電話行銷執行長紀坤傑指出，從疫情爆發開始，抗疫產品和保健食品等需求都非常大。東森購物始終秉持顧客至上的服務精神，除持續關心疫情現況，也積極配合政府政策，疫情期間因國內農漁產外銷受到衝擊，因此率先響應農委會協助開賣鳳梨釋迦、午仔魚與石斑魚力挺小農，另外也攜手新北市政府推出「新北專區」，幫業者拓展虛擬通路商機，更不斷精進服務品質，為台灣社會盡一份心力，也持續為所有會員帶來最愉悅舒適的購物體驗。
▲▼疫情期間，東森購物率先響應農委會政策，於旗下電視購物與購物網等通路大力促銷國內優質農漁產品，台東鳳梨釋迦一開賣立即狂銷3萬台斤，午仔魚與石斑魚2小時熱賣8,000條。（上圖／記者徐文彬攝、下圖／東森購物提供）
----------------------------------------------------------------------

[57] 陸84艘輪船赴沙國搶原油　網嘆：像雙11...真相出爐了

▲國際油價持續下跌。（圖／路透社）
記者曾俊豪／綜合報導
大陸網站近日流傳一篇文章，表示有84艘中國輪船啟航，前往沙國購買原油。不過，文章經媒體查證，並非事實。業內人士分析，國際油價暴跌之時，大陸煉廠從海外進口原油，理論上有利於降低原油進口成本。但加大進口量仍有其他因素限制，搶油對煉廠的提振作用並不會很快顯現。《澎湃》新聞報導，這篇以《中國84艘巨輪拔錨起航 全球最大船隊直撲海灣 外媒感嘆實力懸

▲美國頁岩油有一定市佔率，沙烏地對俄羅斯打石油戰也會連帶影響美國，川普已考慮介入油市。（圖／路透社）
中央社／記者尹俊傑
美國油價今天飆漲近24%，從18年低點回升，創史上最大單日漲幅。面對沙烏地阿拉伯與俄羅斯增產競爭，美國總統川普考慮介入調停，協助油價擺脫昨天大跌陰影。
在2019年冠狀病毒疾病（COVID-19，武漢肺炎）疫情蔓延、沙烏地阿拉伯與俄羅斯油價戰雙重打擊下，國際油價近期走勢淒慘。紐約西德州中級原油（WTI）價格昨天暴跌24.4%，創1991年波灣戰爭爆發以來第2大單日跌幅，收盤價創2002年2月以來新低。
為緩和低油價對美國業者的衝擊，美國能源部今天奉川普指示，宣布將招標採購3000萬桶原油作為戰略儲備，刺激美國油價止跌回升。
此外，華爾街日報引述知情人士說法報導，川普政府考慮介入沙烏地阿拉伯與俄羅斯油價戰，透過外交施壓沙國減產，並威脅對俄羅斯實施制裁，以利恢復油價穩定。
WTI今天飆漲4.85美元，漲幅23.8%，收在每桶25.22美元，創史上最大單日漲幅。倫敦布倫特油價跟漲3.59美元，漲幅14.4%，收在每桶28.47美元。
沙烏地阿拉伯與俄羅斯3月上旬因減產談不攏而撕破臉，沙國隨後放棄支撐油市，改採削價競爭策略，與俄羅斯都打算在限制供油3年協議到期後增加產油。兩國油價戰愈演愈烈之際，武漢肺炎全球疫情惡化，油市前景烏雲密布，導致國際油價不斷下探。
川普今天在白宮記者會表示，油價崩盤雖然讓業者受苦，但消費者會因加油成本降低而受益。他對調停沙烏地阿拉伯與俄羅斯爭端「有點掙扎」，但會在「適當時機」干預油市。
受疫情削弱能源需求衝擊，國際油價今年已崩跌逾半。華爾街日報報導，美國石油業近期遊說川普政府加強外交干預，政府與業界人士都認為，唯有採取外交行動才能讓沙烏地阿拉伯與俄羅斯各退一步，防止油市供給過剩。
知情官員透露，美國將透過國務院、白宮國安會管道直接和沙烏地阿拉伯交涉，要求沙國放棄每日產油量提升至1230萬桶的目標，改採原先降低產量的計畫。
官員說，美國也考慮對俄羅斯祭出制裁，但制裁細節不明，俄羅斯是否會還擊也不清楚。川普政府與沙烏地阿拉伯交涉時，可能提及美國以制裁威脅俄羅斯，讓利雅德相信莫斯科不會因沙國減少產油而輕易獲益。
--------------------------------------------------------------

▲金管會證期局今天宣布，要求證券商應強化相關防疫措施。（圖／資料照）
記者紀佳妘／台北報導
新冠肺炎（COVID-19）疫情在各國蔓延，台灣確診案例累計破百，繼金管會保險局祭出3項出國加強管控措施，證期局今（19）日也宣布，要求證券商應強化相關防疫措施，應掌握員工出國狀況，若回台應依規定進行居家檢疫或自主健康管理。
因應疫情發展，證期局表示，要求證券商應再強化相關防疫措施，包括增加要求業者應宣導員工及各級主管應避免非必要的公務及非公務出國，並應掌握公司人員出國狀況及採行相關防疫管理措施。
證期局指出，若有人員自國外返台，應確實依規定要求進行居家檢疫或自主健康管理。
為提升證券商重視防疫及落實執行，證期局說，17日要求各證券商應在20日前通報各公司的防疲計畫與執行情形，同時也應在證期局每周的疫情調查表中，增加填報公司人員出國狀況及防疫控管資訊。
這份疫情調查表內容，包括查填已出國及未來一定期間預計出國的人數、職務、出國地區、出國期問、回國後執行防疫措施等内容，以利證期局掌握資訊及督促業者落實控管執行。
此外，保險局在16日率先對保險業祭出3項出國加強管控措施，董事長及總經理若有出國規劃，需事先通報，同時也要求保險公司設置「防疫長」，且層級拉高至總經理，並成立「防疫長群組」督促公司確實落實防疫工作。
----------------------------------------------------------------------

[80] 央行經濟成長預期未「保2」　祭2000億融通方案護中小企業

▲ 央行總裁楊金龍。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
全球受到新冠肺炎(COVID-19)疫情干擾，衝擊供應鏈與最終需求，央行今（19）日舉行理監事會議，預估全年經濟成長率未「保2」，為1.72%。
央行表示，隨著肺炎疫情擴散，預期今年國際景氣明顯趨緩，不利台灣出口，加上來台旅客劇減，抑制服務輸出成長。內需方面，疫情影響國內消費信心及就業市場，預期民間消費支出保守；另外，廠商投資計畫亦可能受影響，且去(2019)年資本設備進口大幅擴增，墊高比較基期，民間投資成長將趨溫和。
央行指出，由於疫情不確定性升高，國際機構陸續下修全球經濟成長率，央行預測疫情影響主要在上半年，下半年成長可望逐漸回溫，全年經濟成長率為1.92%，明顯低於上年之2.71%，亦低於央行

▲聯電今日內部公告，所有外國訪客一律禁止進入廠區。（圖／記者周康玉攝）
記者周康玉／新竹報導
台積電（2330）昨（18）日公告一名員工確診新冠肺炎（COVID-19）後，明（20）日將開始實施分流上班，竹科大廠為防疫出奇招，聯電今（19）日宣布，禁止所有外國訪客進入公司；半導體封測廠南茂設置貨櫃屋，讓客戶或供應商可「戶外開會」；網通廠智邦則是在門口搭遮雨棚架，接待訪客；竹科館領局也帶頭示範，大型會議採梅花座。
根據聯電（2303）內部文件指出，因應疾管署昨（18）日最新宣布因國際疫情持續升溫，境外移入個案暴增，今日凌晨零時起，非本國籍人員一律限制入境。不分國籍，所有入境者一律居家檢疫14天。
聯電表示，不僅所有國外訪客不准進入公司，相關裝機、維修等行程盡量不做安排，先確保公司員工健康安全為優先。至於分流辦公，聯電表示，分區辦公會是下一階段的措施，一旦政府宣布有社區感染，就馬上啟動該機制。
由於竹科染疫升溫，竹科管理局帶頭做示範，像是記者會或大型會議地點皆改到通風良好、有窗戶、較為空曠的會議室，座位也採梅花座。
封測廠南茂（8150）目前禁止供應商和客進入廠區；針對內部員工，也減少因公出差。針對訪客，南茂在門口承租貨櫃屋作為替代方案，作為臨時接待訪客區，由各部門預約時段，該時段使用完畢就清潔消毒。
網通大廠智邦（2345）則在廠區入口架起戶外帳篷，並擺設桌椅，讓客戶、供應商等訪客不需進入廠區，即可在戶外訪客區洽談相關事宜。
▼竹科大廠防疫出奇招，南茂承租貨櫃屋戶外開會，下同。（圖／記者周康玉攝）
▼竹科大廠防疫出奇招，智邦科技架外棚會客。（圖／記者周康玉攝）
----------------------------------------------------------------------

[91] 財訊／「中國式復工」　一齣上下呼攏的荒唐劇

文／蔣清平
「快快快，把燈全打開，車間的馬達轟轟轟聲音要出來！」三月初的杭州蕭山經濟技術開發區一名台商對著廠區領班大喊。他邊喘氣邊無奈地說，中央聯合檢查組來稽查復工情況，現在查得嚴，「我們復工了，但根本沒復產」。
北京一名官員透露，自武漢肺炎疫情爆發以來，中南海關注的一直都是經濟，而非醫療，「死人、病人無法抗議，但經濟崩潰，中共也準備垮台了」；因此，一月底各省市紛紛自掃門前雪，推出封閉式管理後，中國經濟停滯，惹得中

文／洪寶山
雪崩式大跌後市場信心不容易立刻恢復，投資人千萬不要隨便心動手癢見獵心喜，記取歷史教訓等盤勢落底確實止穩後再出手，才能反敗為勝。
將來後人寫近二十年的金融史，必然會提到三次重要的股市牛熊分水嶺：二○○一年的九一一恐怖攻擊事件、二○○八年金融海嘯、二○二○年COVID-19新冠病毒。三大利空都造成股市大跌。道瓊指數打噴嚏，全球股市跟著跳水也跌入技術性熊市。逼得美國聯準會短短十三天之內連降六碼，直接零利率，川普更推出一兆美元的財政政策與啟動「直升機撒錢」。
利多一波又一波，多頭的中期反彈要來了嗎？以古鑑今，回顧九一一恐怖攻擊是美國本土首度受到攻擊，重創美國人的安全感，當時正處在聯準會降息週期，事發後聯準會降息兩碼，並持續降息至二○○三年七月的一％。二○○八年的金融海嘯不僅零利率，更三度量化寬鬆。九一一恐怖事件降息二十二碼，金融海嘯降息二十一碼，外加三次量化寬鬆(相當於六碼)，那麼這次瘟疫讓聯準會已經降息六碼，直接歸零，而鮑爾還強調美國不適合負利率。
市場預期央行之間的降息梭哈啟動，金融市場陷入大混亂，投資人採取現金為王的策略將造成金融商品流動性劇降，在新秩序還沒重建之前，單日震盪幅度擴大，以為抄底機會來臨，結果進場才知道是死在半山腰。當下投資人要思考的是，不止損，自己的口袋夠不夠深？止損，能不能管住自己容易衝動的心，遠離盤面不搶短？
這次疫情初期全球以為武漢封城就阻止了新冠病毒擴散，卻選擇性的遺忘武漢封城前一晚三十萬人大逃離的危險，雖然台灣擋下了，日韓歐美卻先後在一個月後成為重災區，封城已成為新常態，經濟活動瞬間劇停。各國央行會大動作降息，不外乎今年全球經濟衰退機率高達九○％，而且在疫苗研發出來之前，經濟衰退還會持續，全球供應鏈與消費力瞬間急煞車，將來重新開機也需要時間。
美股十一年的牛市僅用十九個交易日就結束，這波來得又急又快的熊市，考驗高負債企業的存活能力，也考驗高槓桿投資人的風險控制能力，期指斷頭之後就是融資斷頭，接著就是企業暴雷，海水退潮後才知誰在裸泳。
